# Exploring Chicago Police Data
Bohan Yin

In this project, from Chicago Police Database, I selected several files for analysis: the files including complaints on officers from 2000 to 2016, including accused officers, victims and investigators, and the file including award records that officers received from 1967 to 2017. This assignment analyzed the proportion of complains that are sustained according to different categories, and also allows users to check the background of accused officers. In particular, users can check the background of accused officers by looking at the plot that shows the amount of awards they received as well as the amount of complaints they received.

In [20]:
### Required packages ###
import os, gzip
import pandas as pd
import numpy as np
import bokeh
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from ipywidgets import interact, interact_manual
from math import pi
import warnings
warnings.filterwarnings("ignore")

In [21]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [22]:
### Read in datasets ###
dir_name =  './'
pairs = [('fully-unified-data/complaints/complaints-accused_2000-2016_2016-11.csv.gz', 'df_accused'),
         ('fully-unified-data/complaints/complaints-victims_2000-2016_2016-11.csv.gz', 'df_victims'),
         ('fully-unified-data/complaints/complaints-investigators_2000-2016_2016-11.csv.gz', 'df_investigators'),
         ('discipline_penalty_codes.csv', 'df_penCode'),
         ('fully-unified-data/awards/awards_1967-2017_2017-08.csv.gz', 'df_awards')]

def readin(pathlist):
    raw_data = {}
    for p in pathlist:
        path = os.path.join(dir_name, p[0])
        if p[0].endswith('gz'):
            with gzip.open(path) as f:
                raw_data[p[1]] = pd.read_csv(f)
        elif p[0].endswith('csv'):
            raw_data[p[1]] = pd.read_csv(path, index_col=0)
    return raw_data

raw_data = readin(pairs)
df_accused = raw_data['df_accused']
df_vict = raw_data['df_victims']
df_inv = raw_data['df_investigators'] 
df_pen = raw_data['df_penCode']
df_award = raw_data['df_awards']

# Merge data

In [6]:
### Data Modification ###
def merge_crid(df_x, df_y, left_on = -1, right_on = -1):
    if left_on == right_on == -1: ## this is used when both data frames have the cr_id column
        df_merge = pd.merge(df_x,df_y, how = 'outer', on = 'cr_id')
        assert(all(df_x.loc[df_x['cr_id'].isnull()]))
        assert(all(df_y.loc[df_y['cr_id'].isnull()]))
    else: # this is used when  both data frames don't have cr_id column, so it needs to specify which columns need to merge
        df_merge = pd.merge(df_x,df_y, left_on = left_on, right_on = right_on, how = 'outer')

    return df_merge

## Selecting useful variables for future analysis
def modify(df):
    # since UID is the unique id for each officer and it helps us to identify different officers under same cr_ids, 
    # I think it's important to keep that
    df = df[['cr_id', 'complaint_category', 'final_discipline', 'final_finding', 'UID_x', 'UID_y', 'gender','age', 'race', 'ACTION_TAKEN', 'NOTES']]
    df = df.rename(columns = {'UID_x': 'Accused_UID', 'UID_y': 'Investigator_UID'})
    return df

In [7]:
## Merge with other data
merged_df = merge_crid(merge_crid(merge_crid(df_accused, df_inv), df_vict), df_pen, left_on = 'final_discipline', right_on = 'CODE')
merged_df = modify(merged_df)


# Plot the information of accused officers (the ammount of complaints they received and amount of awards they received)

In [8]:
## Functions that modifies award csv files and accused csv files
def select(df, award = True):
    if award:
        ## Get award info 
        df_award = df[df.current_award_status == 'FINAL']
        df_award = df_award.rename(columns = {'UID':'Accused_UID'})

        df_award1 = df_award.groupby('Accused_UID').award_type.count()
        df_award1 = pd.DataFrame(df_award1)
        df_award1.columns = ['award_numbers']
        return df_award1
    else:
        ## Get accused info
        df_accused = df.groupby('Accused_UID').cr_id.count()
        df_accused = pd.DataFrame(df_accused)
        return df_accused
    
    
def merge_data(df1, df2):    
    new_result = pd.merge(df1, df2, on = 'Accused_UID', how = 'outer').reset_index()
    new_result[np.isnan(new_result)] = 0
    return new_result

In [9]:
awarded = select(df_award)
accused =select(merged_df, award = False)
total = merge_data(awarded, accused)

In [10]:
## Plot the information of accused officers (the ammount of complaints they received and amount of awards they received)
UID = merged_df.groupby('Accused_UID').count().reset_index().Accused_UID.tolist()
def plt_ac(award1, uid = ''):
    plt.scatter(award1['award_numbers'], award1['cr_id'], alpha = 0.5)
    plt.ylabel('Complaints')
    if uid != '':
        try:
            uid = int(uid)
            if uid in UID:
                p = award1.loc[award1['Accused_UID'] == uid]
                plt.scatter(p['award_numbers'].tolist(), p['cr_id'].tolist(), color='red')
                plt.ylabel('Complaints')
            else:
                print('Wrong ID')
        except:
            print('Wrong ID')
    plt.xlabel('award_numbers')
    plt.ylabel('Complaints')
    plt.show()

In [11]:
## Input an UID which related to a certain officer, you can check how many complaints and awards he received
@interact(uid='')
def textbox(uid=''):
    plt_ac(total, uid)
    print(uid)

interactive(children=(Text(value='', description='uid'), Output()), _dom_classes=('widget-interact',))

From the plot above we can see, even if the officer received many awards from the police department, he is still likely to violate some regulations and get complained by victims. So amount ot recieved awards does not reflect the quality of the officer.

# Exploring Sustained Proportion of Accused Officers

In [14]:
## This is the dataframe including all complaint cases
df_award = df_award.rename(columns = {'UID':'Accused_UID'})
df_award1 = df_award.drop_duplicates(subset='Accused_UID')
df_award2 = df_award1.filter(['award_type', 'Accused_UID'])
merged_df2 = merge_crid(merged_df, df_award2, right_on = 'Accused_UID', left_on = 'Accused_UID')

## Calculating using total accused number


In [15]:
def perc_sus(df, field = -1):
    # Here, since there are issues that one cr_id might have multiple accused cases, it is hard to
    # conclude one result for each cr_id. Here, I choose not to use nunique('cr_id'), because it seems 
    # better to include  all 
    # sustained cases when calculating the proportion of sustained complaints. So I calculate the
    # proportion by putting the sum of all sustained cases as the nominator, and the total number
    # of cases as the denominator. 
    if field == -1:           
        total = df.groupby(['complaint_category','final_finding']).count()
        total = total.reset_index()
        su = total.loc[total['final_finding'] == "SU"]
        rate = su['cr_id'].sum()/total['cr_id'].sum()
        return rate.round(4)
    else:
        field_1 = df.groupby([field,'final_finding']).count()
        field_1 = field_1.reset_index()
        field_2 = field_1.groupby(field).sum()
        field_2 = field_2.reset_index()
        field_3 = field_1.loc[field_1['final_finding'] == "SU"]
        field_df = pd.merge(field_2, field_3, how = 'outer', on = field )
        field_df = field_df[[field, 'cr_id_x', 'cr_id_y']]
        field_df = field_df.rename(columns = {'cr_id_x': 'cr_id_total', 'cr_id_y': 'cr_id_'+ field})
        field_df['proportion_by_' + field] = (field_df['cr_id_' + field]/field_df['cr_id_total']).round(2)
        return field_df


In [16]:
## Proportion of complaints that are sustained, split by different tags/fields
tagList = ['race', 'complaint_category', 'award_type']
@interact(tag = tagList)
def sus(tag):
    a = perc_sus(merged_df2, field = tag)
    @interact(category = a[tag].tolist())
    def sus(category):
        print(a[a[tag] == category]["proportion_by_"+tag])

interactive(children=(Dropdown(description='tag', options=('race', 'complaint_category', 'award_type'), value=…

In [17]:
## Interactive Barchart
def pct_sustained_barchart(merged_df, field):
    pct_by_race = perc_sus(merged_df, field) 
    p = figure(x_range=pct_by_race[field], plot_height=250, title="Percent sustained by " + field)
    p.vbar(x=pct_by_race[field], top=pct_by_race['proportion_by_' + field], width=0.9)
 
    p.xgrid.grid_line_color = None
    p.y_range.start = 0

    show(p)

In [18]:
## Different complaint category's proportion sustained, splitted by vitcims' race
categories = merged_df2.complaint_category.unique()
@interact(Value=categories.tolist())
def dropdown(Value=categories.tolist()):
    filtered_df = merged_df2.loc[merged_df2['complaint_category'] == Value]
    pct_sustained_barchart(filtered_df, 'race')

interactive(children=(Dropdown(description='Value', options=('01A-USE OF PROFANITY', '05A-ARRESTEE - DURING AR…

In [19]:
perc_sus(merged_df, field = 'race')

,race,cr_id_total,cr_id_race,proportion_by_race
0,ASIAN/PACIFIC ISLANDER,240,26,0.11
1,BLACK,22315,513,0.02
2,HISPANIC,4508,197,0.04
3,NATIVE AMERICAN/ALASKAN NATIVE,18,3,0.17
4,WHITE,2982,201,0.07
